# Rigid registration

This example should show you how to perform rigid registration between two SIRF images.

SIRF's registration/resampling functionality is provided by wrapping and extending the [NiftyReg](http://cmictig.cs.ucl.ac.uk/wiki/index.php/NiftyReg) code base. Rigid and affine registrations are performed using NiftyReg's symmetric `aladin` algorithm, whereas non-rigid registrations use the symmetric `f3d` algorithm.

Although the example of rigid registration is given here, it is trivial to modify it for affine registrations and only slightly trickier to extend to non-rigid registration.

The images to be registered in this example are `test.nii.gz` and `test2.nii.gz`, which are two T1-weighted MR brain scans taken one year apart.

N.B.: Registration packages use different names for the sets of images they are registering. In NiftyReg (and therefore SIRF), the floating image is moved to match the reference image. In other packages, the floating=moving and reference=fixed. 

## Copyright

Author: Richard Brown
First version: 3rd April 2019

CCP PETMR Synergistic Image Reconstruction Framework (SIRF)
Copyright 2019 University College London.

This is software developed for the Collaborative Computational Project in Positron Emission Tomography and Magnetic Resonance imaging (http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.


## Initial set up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
# Standard stuff
import numpy
import matplotlib.pyplot as plt

# SIRF stuff
from sirf.Utilities import examples_data_path
import sirf.Reg as Reg
examples_path = examples_data_path('Registration')

In [ ]:
#%% First define some handy function definitions
# To make subsequent code cleaner, we have a few functions here. You can ignore
# ignore them when you first see this demo.
# They have (minimal) documentation using Python docstrings such that you 
# can do for instance "help(imshow)"
#
# First a function to display an image

def imshow(image, title=''):
    """Display an image with a colourbar, returning the plot handle. 
    
    Arguments:
    image -- a 2D array of numbers
    limits -- colourscale limits as [min,max]. An empty [] uses the full range
    title -- a string for the title of the plot (default "")
    """
    plt.title(title)
    bitmap=plt.imshow(image)
    limits=[numpy.nanmin(image),numpy.nanmax(image)]
                
    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

## Engine for loading images

By default this example uses `sirf.Reg` as the engine to open the images, which handles NIfTI images. 

You might want to register different types of images - perhaps your floating image is a STIR interfile? If so, change the second line such that it reads `import sirf.STIR as eng_flo`.

In [ ]:
# In this example, we will use sirf.Reg as the engine to open our images
import sirf.Reg as eng_ref
import sirf.Reg as eng_flo

## Open and display the images

In [ ]:
# Open the images
ref_file = examples_path + "/test.nii.gz"
flo_file = examples_path + "/test2.nii.gz"
ref = eng_ref.ImageData(ref_file)
flo = eng_flo.ImageData(flo_file)

In [ ]:
# Display the images
ref_slice = ref.get_dimensions()[1] // 2
flo_slice = flo.get_dimensions()[1] // 2
plt.subplot(1,2,1)
imshow(ref.as_array()[ref_slice,:,:], 'Reference image, slice: %i' % ref_slice)
plt.subplot(1,2,2)
imshow(flo.as_array()[flo_slice,:,:], 'Floating image, slice: %i' % flo_slice);

## Set up the registration object

In [ ]:
# Set to NiftyF3dSym for non-rigid
algo = Reg.NiftyAladinSym()

# Set images
algo.set_reference_image(ref)
algo.set_floating_image(flo)

# What else can we do?
help(algo)

## Setting parameters

From the help above, it looks like we can set registration parameters both via a file and directly. Let's try both.

In [ ]:
# Setting via parameter file
par_file = examples_path + "/paramFiles/niftyreg_aladin.par"
algo.set_parameter_file(par_file)

algo.set_parameter('SetPerformRigid','1')
algo.set_parameter('SetPerformAffine','0')
#algo.set_parameter('SetWarpedPaddingValue','0')  # NaN by default, uncomment to set to 0.

## Register!

In [ ]:
algo.process()

## Show the registered image

The registered image will be the same size as the reference image, so we can use `ref_slice`.

In [ ]:
output = algo.get_output()
output_arr = output.as_array()
plt.figure()
imshow(output_arr[ref_slice,:,:], 'Registered image, slice: %i' % ref_slice);

## Show the transformation matrix

In [ ]:
TM = algo.get_transformation_matrix_forward()
print(TM.as_array())

## Displacement field image
The displacement field image maps the voxels of the floating image to the final warped image. They are particularly interesting for non-rigid registrations, where transformation matrices no longer exist.

Each voxel of the displacement field image contains an (x,y,z) coordinate, so the resulting image is 4D.

(As a small technicality, the NIfTI format stores the time component in the 4th dimension, and the displacement coordinates in the 5th dimension. Therefore the displacement field image is actually a 5D image with a singleton in the 4th dimension.)

In [ ]:
displacement = algo.get_displacement_field_forward()

plt.figure()
for i in range(3):
    plt.subplot(3, 1, 1 + i)
    imshow(displacement.as_array()[ref_slice,:,:,0,i],
           'Displacement field %s-direction, slice: %i' % ("xyz"[i], ref_slice))

# Resampling

Imagine that you got the motion information from another source and wanted to resample an image with it. Easy!

In [ ]:
tm = algo.get_transformation_matrix_forward()

resampler = Reg.NiftyResample()
resampler.set_reference_image(ref)
resampler.set_floating_image(flo)
resampler.add_transformation(tm)
resampler.set_padding_value(0)
resampler.set_interpolation_type_to_linear()
resampler.process()

plt.figure()
imshow(resampler.get_output().as_array()[ref_slice,:,:], 'Resampled image, slice: %i' % ref_slice);